In [1]:
import requests, json, tqdm, base64
import pandas as pd

In [2]:
mkdir data

In [3]:
urls = [
    "http://opendata.auth-6f31f706db6f4a24b55f42a6a79c5086.storage.sbg5.cloud.ovh.net/2019-03-02/LA_TRANSITION_ECOLOGIQUE.json", 
    "http://opendata.auth-6f31f706db6f4a24b55f42a6a79c5086.storage.sbg5.cloud.ovh.net/2019-03-02/LA_FISCALITE_ET_LES_DEPENSES_PUBLIQUES.json",
    "http://opendata.auth-6f31f706db6f4a24b55f42a6a79c5086.storage.sbg5.cloud.ovh.net/2019-03-02/DEMOCRATIE_ET_CITOYENNETE.json", 
    "http://opendata.auth-6f31f706db6f4a24b55f42a6a79c5086.storage.sbg5.cloud.ovh.net/2019-03-02/ORGANISATION_DE_LETAT_ET_DES_SERVICES_PUBLICS.json", 
    "http://opendata.auth-6f31f706db6f4a24b55f42a6a79c5086.storage.sbg5.cloud.ovh.net/2019-03-02/EVENTS.json",
]

# !wget -c https://grandeannotation.fr/download -O actions.zip && unzip actions.zip



--2019-03-14 09:31:28--  https://grandeannotation.fr/download
Resolving grandeannotation.fr (grandeannotation.fr)... 145.239.192.225, 2001:41d0:202:100:145:239:192:225
Connecting to grandeannotation.fr (grandeannotation.fr)|145.239.192.225|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3265560 (3.1M) [application/zip]
Saving to: ‘actions.zip’

actions.zip         100%[===================>]   3.11M  5.40MB/s    in 0.6s    

2019-03-14 09:31:29 (5.40 MB/s) - ‘actions.zip’ saved [3265560/3265560]

Archive:  actions.zip
  inflating: LICENSE.txt             
  inflating: README.txt              
  inflating: actions.csv             


In [4]:
def download_answers(url, path_):
    file_name = url.split('/')[-1]
    data = requests.get(url).json()
    with open("{0}/{1}".format(path_, file_name), "w") as f: 
        json.dump(data, f)

In [5]:
[download_answers(url, "./data") for url in tqdm.tqdm(urls)]

100%|██████████| 5/5 [05:57<00:00, 63.47s/it]


[None, None, None, None, None]

In [6]:
annotations = pd.read_csv("./actions.csv")

In [7]:
def open_answers(path_):
    with open(path_, "r") as f: 
        return json.load(f)

In [8]:
def format_values(json_answers):
    data = []
    for contrib in tqdm.tqdm(json_answers):
        for answer in contrib.get("responses", []):
            tmp_answer = answer
            tmp_answer["reference"]  = contrib["reference"]
            tmp_answer["id"] = contrib["id"]
            data.append(tmp_answer)
            
    return data

In [9]:
transition_eco = format_values(open_answers("./data/LA_TRANSITION_ECOLOGIQUE.json"))
fiscalite = format_values(open_answers("./data/LA_FISCALITE_ET_LES_DEPENSES_PUBLIQUES.json"))
events = format_values(open_answers("./data/EVENTS.json"))
democratie = format_values(open_answers("./data/DEMOCRATIE_ET_CITOYENNETE.json"))
organ = format_values(open_answers("./data/ORGANISATION_DE_LETAT_ET_DES_SERVICES_PUBLICS.json"))

100%|██████████| 67301/67301 [00:01<00:00, 55326.61it/s]


In [166]:
list_of_data = transition_eco + fiscalite + events + democratie + organ

In [167]:
all_contributions = pd.DataFrame(list_of_data) 

In [168]:
clean = all_contributions.dropna(subset=["formattedValue"])

In [169]:
def turn_question_in_id(question_id):
    return base64.b64encode("Question:{0}".format(question_id).encode()).decode()

turn_question_in_id(162)

'UXVlc3Rpb246MTYy'

In [14]:
annotations["question_id"] = annotations.Question.apply(lambda x: turn_question_in_id(x))

In [15]:
clean = clean[clean.questionId.isin(set(annotations.question_id))]

In [16]:
annotations.rename({"Contribution":"reference"}, axis=1, inplace=True)

In [17]:
annotation_with_questions = clean.merge(annotations, 
                                        left_on=["questionId", "reference"],
                                        right_on=["question_id", "reference"], 
                                        how="inner")

In [18]:
annotation_with_questions.head()

,formattedValue,id,questionId,questionTitle,reference,value,Debat,Question,Categorie,Annotateur,Poids,question_id
0,L'Etat,UHJvcG9zYWw6NTk2Nzk5MmItMWUyMS0xMWU5LTk0ZDItZm...,UXVlc3Rpb246MTU3,Et qui doit selon vous se charger de vous prop...,2-7,L'Etat,2,157,Etat,17,1.0,UXVlc3Rpb246MTU3
1,L'Etat,UHJvcG9zYWw6NTk2Nzk5MmItMWUyMS0xMWU5LTk0ZDItZm...,UXVlc3Rpb246MTU3,Et qui doit selon vous se charger de vous prop...,2-7,L'Etat,2,157,Etat,13,1.0,UXVlc3Rpb246MTU3
2,L'Etat,UHJvcG9zYWw6NTk2Nzk5MmItMWUyMS0xMWU5LTk0ZDItZm...,UXVlc3Rpb246MTU3,Et qui doit selon vous se charger de vous prop...,2-7,L'Etat,2,157,Etat,269,1.0,UXVlc3Rpb246MTU3
3,L'Etat,UHJvcG9zYWw6NTk2Nzk5MmItMWUyMS0xMWU5LTk0ZDItZm...,UXVlc3Rpb246MTU3,Et qui doit selon vous se charger de vous prop...,2-7,L'Etat,2,157,Etat,247,1.0,UXVlc3Rpb246MTU3
4,L'Etat,UHJvcG9zYWw6NTk2Nzk5MmItMWUyMS0xMWU5LTk0ZDItZm...,UXVlc3Rpb246MTU3,Et qui doit selon vous se charger de vous prop...,2-7,L'Etat,2,157,Etat,1,1.0,UXVlc3Rpb246MTU3


## Translate to ELF tags

In [27]:
df_lga_to_elf_tags = pd.read_csv("./../raw/matching_tags_lga_elf.csv", sep=";")

In [32]:
df_lga_to_elf_tags.head()

,id_lga,tag_lga,tag_elf
0,149,"Tri sélectif, compost, moins de déchets","Tri sélectif, compost, réduction déchets"
1,149,Transports doux ou moins nombreux,Transports plus verts ou réduits
2,149,"Consommation vertueuse (bio, circuit-court, lo...","Consommation réduite ou vertueuse (bio, circui..."
3,149,"Réduction consommation énergie, eau","Réduction consommation énergie, eau"
4,149,"Réduction consommation alimentaire, vestimentaire",Régime alimentaire


In [34]:
matching = {row["tag_lga"]:row["tag_elf"] for r, row in df_lga_to_elf_tags.iterrows()}

In [57]:
matching.get("orf", "orf")

'orf'

In [73]:
annotation_with_questions["elf_tag"] = annotation_with_questions.Categorie.apply(lambda x: "Inciter à se présenter" if x == "Inciter à se présenter" else matching.get(x, x))

In [75]:
df_agg = annotation_with_questions.groupby(["reference", "question_id"]).agg({
    "elf_tag": lambda x: set(x.tolist())
}).reset_index()

In [76]:
df_agg.head()

,reference,question_id,elf_tag
0,1-100,UXVlc3Rpb246MTA3,"{Société civile, citoyens, Z Autres}"
1,1-10000,UXVlc3Rpb246MTE2,{vote obligatoire}
2,1-10002,UXVlc3Rpb246MTA3,{A Moi-même ou personne}
3,1-10004,UXVlc3Rpb246MTA3,"{Société civile, citoyens, Tous les élus, X Si..."
4,1-10004,UXVlc3Rpb246MTEz,{Q Proportionnelle}


In [77]:
df_grouped = df_agg.merge(annotation_with_questions.drop_duplicates(subset=["reference", "question_id"]),
                          left_on=["reference", "question_id"], 
                          right_on=["reference", "questionId"], 
                         how="left")


In [78]:
df_grouped.head()

,reference,question_id_x,elf_tag_x,formattedValue,id,questionId,questionTitle,value,Debat,Question,Categorie,Annotateur,Poids,question_id_y,elf_tag_y
0,1-100,UXVlc3Rpb246MTA3,"{Société civile, citoyens, Z Autres}",Le peuple,UHJvcG9zYWw6ZmQ2OTFiNDMtMWUyNi0xMWU5LTk0ZDItZm...,UXVlc3Rpb246MTA3,En qui faites-vous le plus confiance pour vous...,Le peuple,1,107,"Société civile, citoyens",406,1.054380,UXVlc3Rpb246MTA3,"Société civile, citoyens"
1,1-10000,UXVlc3Rpb246MTE2,{vote obligatoire},En rendant le vote obligatoire,UHJvcG9zYWw6MWJjMTBiNjEtMWYyYS0xMWU5LTk0ZDItZm...,UXVlc3Rpb246MTE2,Que pensez-vous de la participation des citoye...,En rendant le vote obligatoire,1,116,Vote obligatoire (sanction),13,1.267280,UXVlc3Rpb246MTE2,vote obligatoire
2,1-10002,UXVlc3Rpb246MTA3,{A Moi-même ou personne},moi même,UHJvcG9zYWw6MWQ2NThkYWItMWYyYS0xMWU5LTk0ZDItZm...,UXVlc3Rpb246MTA3,En qui faites-vous le plus confiance pour vous...,moi même,1,107,A Moi-même ou personne,2,1.000145,UXVlc3Rpb246MTA3,A Moi-même ou personne
3,1-10004,UXVlc3Rpb246MTA3,"{Société civile, citoyens, Tous les élus, X Si...",Représentation des français,UHJvcG9zYWw6MjRmY2E5ZTEtMWYyYS0xMWU5LTk0ZDItZm...,UXVlc3Rpb246MTA3,En qui faites-vous le plus confiance pour vous...,Représentation des français,1,107,Sans réponse / Hors sujet / Inclassable,811,16.365496,UXVlc3Rpb246MTA3,Sans réponse / Hors sujet / Inclassable
4,1-10004,UXVlc3Rpb246MTEz,{Q Proportionnelle},Proportionnelle,UHJvcG9zYWw6MjRmY2E5ZTEtMWYyYS0xMWU5LTk0ZDItZm...,UXVlc3Rpb246MTEz,Que faudrait-il faire pour mieux représenter l...,Proportionnelle,1,113,Q Proportionnelle,18,1.000000,UXVlc3Rpb246MTEz,Q Proportionnelle


In [79]:
#Remove extra characters at the begining of LGA tags
def clean_tags(tags):
    return [tag if len(tag.split()[0])>1 else " ".join(tag.split()[1:]) for tag in tags if tag]
    

In [80]:
train_data_lga = [{
    "question": row["questionTitle"],
    "answer": row["formattedValue"],
    "target": clean_tags(row.elf_tag_x)
} for  r, row  in df_grouped.iterrows()]

In [82]:
with open("data/train_data_lga.json", "w") as f:
    json.dump(train_data_lga, f)

# Extract ELF Data

In [159]:
df_elf_tags = pd.read_csv("./data/elf_tags_sample_v5.csv", sep=";")
df_elf_questions = pd.read_csv("./../raw/questions_ELF.csv", sep=",")

In [160]:
df_elf = df_elf_tags.merge(df_elf_questions, how="left", left_on="question", right_on="id_elf")

In [161]:
df_elf.shape

(23125, 9)

In [162]:
df_elf_grouped = df_elf.groupby(["question", "contribution"]).agg({
    "tag": lambda x: set(x.tolist()),
    "corpus":"first",
    "question_elf":"first"
}).reset_index()

In [164]:
train_data_elf = [{
    "question": row["question_elf"],
    "answer": row["corpus"],
    "target": clean_tags(row.tag)
} for  r, row  in df_elf_grouped.iterrows()]

# Compose the two 

In [170]:
full_train_data = train_data_lga + train_data_elf

In [171]:
with open("data/all_train_data.json", "w") as f:
    json.dump(full_train_data, f)

In [172]:
with open("data/train_data_lga.json", "w") as f:
    json.dump(train_data_lga, f)

In [173]:
with open("data/train_data_elf.json", "w") as f:
    json.dump(train_data_elf, f)